In [50]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, AzureChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
from supabase import create_client
#from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.output_parsers import StrOutputParser
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.schema import Document
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Literal
from typing import List
from typing_extensions import TypedDict
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from pprint import pprint
from typing import Any,  Literal, Union, Optional,Callable
from langchain_core.messages import  AnyMessage
from langchain.schema import AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import ToolMessage
import logging
import os

In [51]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, AzureChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
from supabase import create_client
#from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.output_parsers import StrOutputParser
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.schema import Document
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Literal
from typing import List
from typing_extensions import TypedDict
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from pprint import pprint
from typing import Any,  Literal, Union, Optional,Callable
from langchain_core.messages import  AnyMessage
from langchain.schema import AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import ToolMessage
import logging
import os

In [52]:
import load_data

In [53]:
from prompts import PRIMARY_ASSISTANT_PROMPT, FEEDBACK_TOOL_PROMPT,FEEDBACK_AGENT_SYSTEM_PROMPT,RAG_AGENT_SYSTEM_PROMPT

In [54]:
from tools import CompleteOrEscalate, toFeedbackAssistant,toConceptualAssistant

In [ ]:
from langchain.callbacks.base import BaseCallbackHandler

class StreamHandler(BaseCallbackHandler):
    def __init__(self):
        self.response = ''

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        # Append the new token to the response
        self.response += token
        # Optionally, update your UI or print the token

In [55]:
stream_handler = StreamHandler()
chat= AzureChatOpenAI(azure_deployment="gpt-4o-rfmanrique",streaming=True,callbacks=[stream_handler])





In [56]:
prompt="Eres un agente conversacional encargado de responder las preguntas del ususario  : {user_input}"

In [57]:
prompt2 = ChatPromptTemplate.from_messages(
    [
        ("system", prompt),      
        
    ]
)

In [58]:
agent=prompt2|chat

In [61]:
memory = MemorySaver()

In [62]:
class State(TypedDict):
    messages:Annotated[list,add_messages]



graph_builder=StateGraph(State)

In [63]:
memory = MemorySaver()

In [64]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
OPENAI_API_VERSION=os.getenv("OPENAI_API_VERSION")

In [67]:
def primary_assistant(state):
    stream_handler.response=''
    message=agent.stream({"user_input":state['messages'][-1]})

    
    
    return {"messages": [AIMessage(content=stream_handler.response)]}

In [68]:
graph_builder.add_node("assistant",primary_assistant)

In [69]:
graph_builder.add_edge(START,"assistant")

In [70]:
graph_builder.add_edge("assistant",END)

In [71]:
graph=graph_builder.compile(checkpointer=memory)

In [72]:
config={"configurable":{"thread_id":50}}

In [73]:
lista=["hola", "Explicame qué es pandas y para qué sirve", "Gracias"]
for i in lista:
    for event in graph.stream({"messages": ("user", i)},config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

Assistant: 
Assistant: 
Assistant: 
